<a href="https://colab.research.google.com/github/aliprg0/Google_Colab/blob/main/Binary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
!pip install yfinance
!pip install yahooquery
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,LSTM
from tensorflow.keras.layers import Dropout
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
from sklearn.tree import DecisionTreeClassifier

In [80]:
#with open("syms.txt","r") as f:
#  lines = f.readlines()
#  f.close()
#lst = []
#for line in lines:
#  line = line.strip()
#  lst.append(line)

s = Screener()
data = s.get_screeners('all_cryptocurrencies_us', count=250)
dicts = data['all_cryptocurrencies_us']['quotes']
symbols = [d['symbol'] for d in dicts]


In [81]:
#symbols = ["MSFT","AAPL","GOOG","TSLA","GGPI","AI","AMZN"]
#symbols = ["BTC-USD","LTC-USD","TRX-USD","XRP-USD","ETH-USD","BNB-USD","DASH-USD","VET-USD","LINK-USD","ADA-USD","DOT-USD","SOL-USD","BCH-USD","FTT-USD","FIL-USD","XMR-USD"]
#symbols = ["AAPL","MSFT","TSLA","GOOG"]
#symbols = ["BTC-USD","ETH-USD"]
#symbols = ["BTC-USD"]

dfs = []
#symbols = lst[:15]
for i in symbols:
   data = yf.download(i,period="Max",interval="1d")
   if data.empty :
     print("Passing...")
   else:
        dfs.append(data)
ndfs = dfs
dfs = []
for df in ndfs:
    if df.shape[0] != 1 & df.shape[0] != 0:
       dfs.append(df)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [86]:
fixed_dfs = []
for df in dfs:
    fixed_df = []
    df = np.array(df)
    for i in range(15, df.shape[0]-1):
        fifteen_days_ago = i-15
        fourteen_days_ago = i-14
        thirteen_days_ago = i-13
        twelve_days_ago = i-12
        eleven_days_ago = i-11
        ten_days_ago = i-10
        nine_days_ago = i-9
        eight_days_ago = i-8
        seven_days_ago = i-7
        six_days_ago = i-6
        five_days_ago = i-5
        four_days_ago = i-4
        three_days_ago = i-3
        two_days_ago = i - 2
        yesterday = i - 1
        today = i
        tomorrow = i + 1
        if df[tomorrow][3] > df[today][3]:
            future = 1
        else:
            future = 0


        row = [df[fifteen_days_ago][3],df[fourteen_days_ago][3],df[thirteen_days_ago][3],df[twelve_days_ago][3],df[eleven_days_ago][3],
                   df[ten_days_ago][3], df[nine_days_ago][3], df[eight_days_ago][3], df[seven_days_ago][3], df[six_days_ago][3],
                   df[five_days_ago][3], df[four_days_ago][3], df[three_days_ago][3], df[two_days_ago][3], df[yesterday][3],
                   df[today][3], future]

        fixed_df.append(row)


    arrayed_fixed_df = np.array([fixed_df])
    two_d_fixed_df = []
    for i in range(0, arrayed_fixed_df.shape[1]):
       two_d_fixed_df.append(arrayed_fixed_df[0][i])
    
    mm = np.array(two_d_fixed_df)


    column_names = ["fifteen_days_ago","fourteen_days_ago","thirteen_days_ago","twelve_days_ago","eleven_days_ago",
                      "ten_days_ago", "nine_days_ago", "eight_days_ago", "seven_days_ago", "six_days_ago",
                       "five_days_ago", "four_days_ago", "three_days_ago", "two_days_ago", "yesterday", 
                        "today", "result"]
    
    column_names_without_result = ["fifteen_days_ago","fourteen_days_ago","thirteen_days_ago","twelve_days_ago","eleven_days_ago",
                      "ten_days_ago", "nine_days_ago", "eight_days_ago", "seven_days_ago", "six_days_ago",
                       "five_days_ago", "four_days_ago", "three_days_ago", "two_days_ago", "yesterday", 
                        "today"]
    dff = pd.DataFrame(mm, columns=column_names)
    
    scaler = MinMaxScaler(feature_range=(0.5,2))
    result = dff["result"]
    dff = dff.drop(["result"],axis=1)
    scaled = pd.DataFrame(scaler.fit_transform(dff.T).T, columns=column_names_without_result)

    scaled["result"] = result
       
    fixed_dfs.append(scaled)

data = fixed_dfs[0]
for i in range(1,len(dfs)):
  data = pd.concat([data,fixed_dfs[i]], ignore_index = True)
data = data.dropna()
X = data.drop(["result"],axis=1)
y = data["result"]
xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.1,random_state=11)
print(xTrain.shape,end=" ")
print(yTrain.shape)
print(xTest.shape,end=" ")
print(yTest.shape)

(73224, 16) (73224,)
(8137, 16) (8137,)


In [87]:
data

,fifteen_days_ago,fourteen_days_ago,thirteen_days_ago,twelve_days_ago,eleven_days_ago,ten_days_ago,nine_days_ago,eight_days_ago,seven_days_ago,six_days_ago,five_days_ago,four_days_ago,three_days_ago,two_days_ago,yesterday,today,result
0,2.000000,1.400197,0.859656,1.116907,0.933050,0.993788,1.607175,1.377677,1.165593,1.035235,0.945795,0.538457,0.507203,0.716479,0.655777,0.500000,0.0
1,1.776787,1.193848,1.471277,1.272998,1.338501,2.000000,1.752501,1.523781,1.383199,1.286743,0.847455,0.813750,1.039441,0.973977,0.805982,0.500000,0.0
2,1.424901,1.622815,1.481366,1.528095,2.000000,1.823437,1.660271,1.559981,1.491171,1.177788,1.153744,1.314749,1.268048,1.148202,0.929918,0.500000,0.0
3,1.650155,1.518958,1.562300,2.000000,1.836235,1.684896,1.591875,1.528053,1.237385,1.215083,1.364418,1.321102,1.209943,1.007482,0.608725,0.500000,1.0
4,1.518958,1.562300,2.000000,1.836235,1.684896,1.591875,1.528053,1.237385,1.215083,1.364418,1.321102,1.209943,1.007482,0.608725,0.500000,0.624509,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81356,1.533053,1.778038,2.000000,1.510863,1.526115,1.400777,1.503930,1.464629,1.241683,1.142863,0.629170,0.748864,0.653861,0.542159,0.520606,0.500000,0.0
81357,1.786378,2.000000,1.529241,1.543920,1.423291,1.522569,1.484744,1.270175,1.175067,0.680676,0.795872,0.704439,0.596933,0.576191,0.556359,0.500000,0.0
81358,2.000000,1.594161,1.606816,1.502822,1.588409,1.555800,1.370821,1.288829,0.862616,0.961927,0.883103,0.790423,0.772540,0.755443,0.706857,0.500000,1.0
81359,1.982850,2.000000,1.859063,1.975054,1.930862,1.680171,1.569052,0.991432,1.126021,1.019196,0.893592,0.869357,0.846187,0.780340,0.500000,0.532415,0.0


In [88]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(xTrain,yTrain)
print(f"Random Forest :  {clf.score(xTest,yTest)}")
logisticRegr = LogisticRegression()
logisticRegr.fit(xTrain, yTrain)
print(f"Logistic Regression: {logisticRegr.score(xTest, yTest)}")
nc = NearestCentroid()
nc.fit(xTrain,yTrain)
print(f"Nearest Centroid: {nc.score(xTest,yTest)}")
gnb = GaussianNB()
gnb.fit(xTrain, yTrain)
print(f"GaussianNB: {gnb.score(xTest,yTest)}")
clf2 = KNeighborsClassifier(n_neighbors=2)
clf2.fit(xTrain, yTrain)
print(f"K-Neighbors: {clf2.score(xTest,yTest)}")
tree = DecisionTreeClassifier()
tree.fit(xTrain, yTrain)
print(f"Decision Tree: {tree.score(xTest,yTest)}")
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
   max_depth=1, random_state=0).fit(xTrain, yTrain)
gb.fit(xTrain,yTrain)
print(f"Gradient Boost: {gb.score(xTest, yTest)}")
svm = SVC()
svm.fit(xTrain,yTrain)
print(f"SVM: {svm.score(xTest,yTest)}")

Random Forest :  0.5876858793167015
Logistic Regression: 0.5305395108762443
Nearest Centroid: 0.5140715251321125
GaussianNB: 0.5127196755561018
K-Neighbors: 0.5855966572446848
Decision Tree: 0.5425832616443407


KeyboardInterrupt: ignored

In [89]:
model = Sequential()
model.add(Dense(256, activation='relu', kernel_initializer='he_normal', input_shape=(16,)))
model.add(Dense(256, activation='relu', kernel_initializer='he_normal',))
model.add(Dense(256, activation='relu', kernel_initializer='he_normal',))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_46 (Dense)            (None, 256)               4352      
                                                                 
 dense_47 (Dense)            (None, 256)               65792     
                                                                 
 dense_48 (Dense)            (None, 256)               65792     
                                                                 
 dense_49 (Dense)            (None, 1)                 257       
                                                                 
Total params: 136,193
Trainable params: 136,193
Non-trainable params: 0
_________________________________________________________________


In [90]:
model.fit(xTrain,yTrain,epochs=50,batch_size=8,validation_data=(xTest,yTest))

Epoch 1/50
9153/9153 [==============================] - 23s 2ms/step - loss: 0.6949 - accuracy: 0.5172 - val_loss: 0.7082 - val_accuracy: 0.5132
Epoch 2/50
9153/9153 [==============================] - 22s 2ms/step - loss: 0.6893 - accuracy: 0.5344 - val_loss: 0.6890 - val_accuracy: 0.5316
Epoch 3/50
9153/9153 [==============================] - 23s 2ms/step - loss: 0.6877 - accuracy: 0.5397 - val_loss: 0.6902 - val_accuracy: 0.5288
Epoch 4/50
9153/9153 [==============================] - 22s 2ms/step - loss: 0.6862 - accuracy: 0.5429 - val_loss: 0.6872 - val_accuracy: 0.5369
Epoch 5/50
9153/9153 [==============================] - 23s 2ms/step - loss: 0.6851 - accuracy: 0.5447 - val_loss: 0.6861 - val_accuracy: 0.5384
Epoch 6/50
9153/9153 [==============================] - 22s 2ms/step - loss: 0.6841 - accuracy: 0.5472 - val_loss: 0.6869 - val_accuracy: 0.5352
Epoch 7/50
9153/9153 [==============================] - 22s 2ms/step - loss: 0.6833 - accuracy: 0.5472 - val_loss: 0.6914 - val_ac